# Light Curves Simulation for all SNe types

In [1]:
import snsim
import importlib
import matplotlib.pyplot as plt
import numpy as np

#secutiry check for numpy parallelization when using dask
def limit_numpy(nthreads=1):
    """ """
    import os
    threads = str(nthreads)
    print(f"threads {threads}")
    os.environ["NUMEXPR_NUM_THREADS"] = threads
    os.environ["OMP_NUM_THREADS"] = threads
    os.environ["OPENBLAS_NUM_THREADS"] = threads
    os.environ["MKL_NUM_THREADS"] = threads
    os.environ["VECLIB_MAXIMUM_THREADS"] = threads
limit_numpy(1)

import dask
import pandas as pd

threads 1


### we use dask to speed up the simulation

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.22 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46571,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.22 GiB
Comm: tcp://127.0.0.1:42907,Total threads: 2
Dashboard: http://127.0.0.1:37265/status,Memory: 3.81 GiB
Nanny: tcp://127.0.0.1:34909,


### init the survey

In [3]:
survey_conf = {
        'survey_file': './survey_file_example.csv',
        'sig_psf': 0.0,
        'sig_zp': 0.01,
        'zp': 25,
        'gain': 1,
        'ra_size': 7.295,
        'dec_size': 7.465,
        'noise_key': ['skynoise', 'skysigADU'], 
         }

### init the generator obejct for all SNe

In [4]:
snia_gen = {
    'M0': 'jla',
    'sigM': 0.1,
    'sct_model': 'G10',
    'force_n': 1000 ,
    'model_name': 'salt2',
    'alpha': 0.14,
    'beta': 2.9,
    'dist_x1': 'N21',
    'dist_c': [-0.055, 0.023, 0.150]}

# for simplicity we use same configuration for SN core collapse
sncc_gen = {'M0': -19,
            'sigM': [1.5,1.5],
            'rate': 'ztf20',
            'model_name': 'vinc_nocorr'
            }

cosmology = {'name':'planck18'}



cmb = {'v_cmb': 0,
       'l_cmb':0,
       'b_cmb':0}

mw_dust = {'model': 'CCM89',}

#distribution of peculiar velocities of SNe
vpec_dist = {'mean_vpec':0,
             'sig_vpec':300}


## configuration dictionary, input of simulation

In [5]:
config_dic = {'data': {'write_path': './',
                       'sim_name': 'Test.simulation.allSN',
                       'write_format': 'parquet'},
              'survey_config': survey_conf,
              'sim_par': {'randseed': 1234, # Optional
                          'z_range': [0.01, 0.3]},
              'snia_gen': snia_gen,
              #'sniipl_gen': sncc_gen,
              #'sniin_gen': sncc_gen,
             # 'sniib_gen': sncc_gen,
             # 'snib_gen': sncc_gen,
              #'snic_gen': sncc_gen,
              #'snic-bl_gen': sncc_gen,
              'cosmology': cosmology,
              'mw_dust': mw_dust,
              'vpec_dist': vpec_dist,
              'cmb': cmb,
              'dask': {'use': True,
                       'nworkers': 4}#parameters to use dask, nworkers depends on your machine
             }

In [6]:
#config dask
cluster.scale(config_dic['dask']['nworkers'])

## Run the simulation

In [7]:
import importlib
importlib.reload(snsim.astrobj)
importlib.reload(snsim.simu)
importlib.reload(snsim.generators)
importlib.reload(snsim.survey_host)
importlib.reload(snsim.sample)

importlib.reload(snsim)


<module 'snsim' from '/home/carreres/Documents/snsim/snsim/__init__.py'>

In [8]:
%%time
#initialize simulator object
#first time a bit long sncosmo has to download all the Templates
sim = snsim.Simulator(config_dic)

CPU times: user 958 ms, sys: 52.9 ms, total: 1.01 s
Wall time: 949 ms


In [9]:
     # -- Generate n base param
param_tmp = sim.generators[0].gen_basic_par(1000, 1234, 
                                            min_max_t=True)


In [10]:
epochs, params = sim.survey.get_observations(
            param_tmp,
            phase_cut=None,
            nep_cut=sim.nep_cut,
            IDmin=0,
            use_dask=sim.config['dask']['use'],
            npartitions=sim.config['dask']['nworkers'])

In [11]:
#run the simulation
sim.simulate()

      _______..__   __.         _______. __  .___  ___. 
     /       ||  \ |  |        /       ||  | |   \/   | 
    |   (----`|   \|  |       |   (----`|  | |  \  /  | 
     \   \    |  . `  |        \   \    |  | |  |\/|  | 
 .----)   |   |  |\   |    .----)   |   |  | |  |  |  | 
 |_______/    |__| \__|    |_______/    |__| |__|  |__| 
================================= Version : 0.4.5+dev ====== 
-----------------------------------------------------------

SIM NAME : Test.simulation.allSN
CONFIG FILE : No config file
SIM WRITE DIRECTORY : ./
SIMULATION RANDSEED : 1234

-----------------------------------------------------------

SURVEY FILE : ./survey_file_example.csv

First day in survey_file : 58000.00 MJD / 2017-09-04 00:00:00.000
Last day in survey_file : 58100.00 MJD / 2017-12-13 00:00:00.000

Survey effective duration is 100.00 days

Survey envelope area is 57.74 squared degrees (0.1 % of the sky)

No cut on survey file.

------------------------------------------------------

### After the simulations

In [12]:
#the sim object has the attribute samples, which contain the data of all the object we simulated
sim.samples
#one sim.sample object for each generator 

In [13]:
sim.samples[0].header
#header with useful parameter of each simulation

{'seed': 1234,
 'seed_key': (0,),
 'obj_type': 'SNIa',
 'rate': 'lambda z: 3e-5',
 'model_name': ['salt2'],
 'model_version': ['T23'],
 'm_vp': 0,
 's_vp': 300,
 'M0_band': 'bessell_b',
 'dist_x1': 'N21',
 'peak_c': -0.055,
 'dist_c': 'asym_gauss',
 'sig_c_low': 0.023,
 'sig_c_hi': 0.15,
 'cosmo': {'cosmod_name': 'planck18'}}

In [14]:
#meta attribute return the parameters of each object, same as sim_lcs.attrs
sim.samples[0].meta[0]

{'mu': 40.28118938167095,
 'zobs': 0.2232167831295382,
 'zCMB': 0.2232167831295382,
 'ID': 0,
 'zcos': 0.2215452437304086,
 'como_dist': 929.2671202887846,
 'zpcmb': 0.0,
 't0': 58034.90950342247,
 'ra': 0.6614847077168625,
 'dec': 0.7580354265768443,
 'vpec': 410.23032726860123,
 'min_t': 58010.44516775988,
 'max_t': 58096.07034257895,
 '1_zobs': 1.2232167831295382,
 'model_name': 'salt2',
 'model_version': 'T23',
 'M0': -19.123830232811475,
 'coh_sct': -0.06105215635050651,
 'x1': 1.1158189250371295,
 'c': -0.01899610101045291,
 'alpha': 0.14,
 'beta': 2.9,
 'G10_RndS': 998124575180,
 'mw_ebv': 0.06874377744226459,
 'mw_r_v': 3.1,
 'mb': 21.307610334944478,
 'x0': 4.750281214636984e-05}

In [15]:
#return the simulated lightcurves
sim.samples[0].sim_lcs

time   fluxtrue  fluxerrtrue         flux     fluxerr  \
ID  epochs                                                                  
0   0       58012.244898   0.248910   242.029564   190.217073  242.428025   
    1       58014.285714   1.198468   107.631635   -65.765801  107.932863   
    2       58016.326531   2.550009    18.352943    17.646399   18.760405   
    3       58018.367347   7.074015   571.817425   276.686076  572.058800   
    4       58020.408163  11.697812   213.912759   -38.402697  213.975435   
...                  ...        ...          ...          ...         ...   
999 33      58091.836735   7.826606   882.053452  2320.115717  883.621645   
    34      58093.877551   2.091887   998.986115   481.458824  999.235852   
    35      58095.918367   2.015024   210.306143  -221.887156  210.838142   
    36      58097.959184   1.942370   903.797867   466.561234  904.065080   
    37      58100.000000   1.864840   535.022071  -710.226112  535.723593   

                  mag    magerr  zp zpsys  gain    skynoise  band  fieldID  \
ID  epochs                                                                   
0   0       19.301876  1.383750  25    ab     1  242.029050  ztfr        1   
    1             NaN       NaN  25    ab     1  107.626067  ztfr        1   
    2       21.883360  1.154278  25    ab     1   18.283325  ztfg        1   
    3       18.895032  2.244800  25    ab     1  571.811236  ztfr        1   
    4             NaN       NaN  25    ab     1  213.885387  ztfg        1   
...               ...       ...  ..   ...   ...         ...   ...      ...   
999 33      16.586226  0.413505  25    ab     1  882.049012  ztfr        1   
    34      18.293602  2.253373  25    ab     1  998.985068  ztfg        1   
    35            NaN       NaN  25    ab     1  210.301351  ztfg        1   
    36      18.327728  2.103853  25    ab     1  903.796792  ztfg        1   
    37            NaN       NaN  25    ab     1  535.020328  ztfg        1   

            sig_zp  sig_psf  
ID  epochs                   
0   0         0.01      0.0  
    1         0.01      0.0  
    2         0.01      0.0  
    3         0.01      0.0  
    4         0.01      0.0  
...            ...      ...  
999 33        0.01      0.0  
    34        0.01      0.0  
    35        0.01      0.0  
    36        0.01      0.0  
    37        0.01      0.0  

[31597 rows x 15 columns]